Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder\
        .appName("rdd_wordcount")\
        .master("spark://spark-master:7077")\
        .config("spark.blockManager.port", "10025")\
        .config("spark.driver.blockManager.port", "10026")\
        .config("spark.spark.driver.host", "spark-master")\
        .config("spark.cores.max", "1")\
        .config("spark.executor.memory", "512m")\
        .getOrCreate()

21/11/18 18:44:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df=spark.read.option('header', True).option('sep',',').option('inferSchema', True).csv('/opt/workspace/owid-covid-data.csv')

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [45]:
df1=df.select('iso_code','location', 'total_cases', 'population').where(df.date=='2021-03-31').filter(~df.iso_code.contains('OWID'))

In [48]:
percent=df1.select('iso_code','location',(df.total_cases/df.population*100).alias ('percent_cases'))
percent.orderBy(percent.percent_cases.desc()).show(15)

+--------+-------------+------------------+
|iso_code|     location|     percent_cases|
+--------+-------------+------------------+
|     AND|      Andorra|15.543907331909661|
|     MNE|   Montenegro|14.523725364693293|
|     CZE|      Czechia|14.308848404077997|
|     SMR|   San Marino|13.937179562732041|
|     SVN|     Slovenia|10.370805779121204|
|     LUX|   Luxembourg| 9.847342390123583|
|     ISR|       Israel| 9.625106044786802|
|     USA|United States| 9.203010995860707|
|     SRB|       Serbia| 8.826328557933492|
|     BHR|      Bahrain| 8.488860079114566|
|     PAN|       Panama| 8.228739065460761|
|     PRT|     Portugal| 8.058699735120369|
|     EST|      Estonia| 8.022681579659551|
|     SWE|       Sweden| 7.969744347858805|
|     LTU|    Lithuania| 7.938864728274825|
+--------+-------------+------------------+
only showing top 15 rows



Задание 2.
Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [49]:
from pyspark.sql.window import Window

In [52]:
df2=df.select('location', 'date', 'new_cases').where((df.date > '2021-03-24')& (df.date <='2021-03-31')).filter(~df.iso_code.contains('OWID'))
df2.show(15)

+-----------+----------+---------+
|   location|      date|new_cases|
+-----------+----------+---------+
|Afghanistan|2021-03-25|     34.0|
|Afghanistan|2021-03-26|     28.0|
|Afghanistan|2021-03-27|     36.0|
|Afghanistan|2021-03-28|      4.0|
|Afghanistan|2021-03-29|     28.0|
|Afghanistan|2021-03-30|     62.0|
|Afghanistan|2021-03-31|     70.0|
|    Albania|2021-03-25|    472.0|
|    Albania|2021-03-26|    449.0|
|    Albania|2021-03-27|    425.0|
|    Albania|2021-03-28|    493.0|
|    Albania|2021-03-29|    285.0|
|    Albania|2021-03-30|    304.0|
|    Albania|2021-03-31|    434.0|
|    Algeria|2021-03-25|    105.0|
+-----------+----------+---------+
only showing top 15 rows



In [53]:
max__cases=df2.withColumn('max_',F.max('new_cases').over(Window.partitionBy("location")))

In [54]:
max__cases.select('location', 'date', 'max_').where( F.col('max_')==F.col('new_cases')).orderBy(max__cases.max_.desc()).show(10)

+-------------+----------+--------+
|     location|      date|    max_|
+-------------+----------+--------+
|       Brazil|2021-03-25|100158.0|
|United States|2021-03-26| 77321.0|
|        India|2021-03-31| 72330.0|
|       France|2021-03-31| 59054.0|
|       Turkey|2021-03-31| 39302.0|
|       Poland|2021-03-26| 35145.0|
|      Germany|2021-03-31| 25014.0|
|        Italy|2021-03-26| 24076.0|
|         Peru|2021-03-25| 19206.0|
|      Ukraine|2021-03-26| 18226.0|
+-------------+----------+--------+
only showing top 10 rows



Зажание 3.
Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [57]:
windowSpec  = Window.partitionBy("location").orderBy("date")
df3=df2.select('location', 'date', 'new_cases').where(df.location=='Russia').withColumn("er_cases", F.lag("new_cases",1).over(windowSpec)) 
#df3=df2.select('location', 'date', 'new_cases').where(df.location=='Russia')
df3.show()

+--------+----------+---------+--------+
|location|      date|new_cases|er_cases|
+--------+----------+---------+--------+
|  Russia|2021-03-25|   9128.0|    null|
|  Russia|2021-03-26|   9073.0|  9128.0|
|  Russia|2021-03-27|   8783.0|  9073.0|
|  Russia|2021-03-28|   8979.0|  8783.0|
|  Russia|2021-03-29|   8589.0|  8979.0|
|  Russia|2021-03-30|   8162.0|  8589.0|
|  Russia|2021-03-31|   8156.0|  8162.0|
+--------+----------+---------+--------+



In [58]:
df3.withColumn('diff', F.col('er_cases')-F.col('new_cases')).show()

+--------+----------+---------+--------+------+
|location|      date|new_cases|er_cases|  diff|
+--------+----------+---------+--------+------+
|  Russia|2021-03-25|   9128.0|    null|  null|
|  Russia|2021-03-26|   9073.0|  9128.0|  55.0|
|  Russia|2021-03-27|   8783.0|  9073.0| 290.0|
|  Russia|2021-03-28|   8979.0|  8783.0|-196.0|
|  Russia|2021-03-29|   8589.0|  8979.0| 390.0|
|  Russia|2021-03-30|   8162.0|  8589.0| 427.0|
|  Russia|2021-03-31|   8156.0|  8162.0|   6.0|
+--------+----------+---------+--------+------+



In [11]:
spark.stop()